# Segmenting and Clustering Neighborhoods in Toronto

importing request library to scrap the wikipedia page

In [220]:
import requests

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')

Of the extracted wikipedia page, only table of class "wikitable sortable" is relevant, so extracting it using beautiful soup

In [221]:
My_table = soup.find('table' ,{'class':'wikitable sortable'})

finding all tags for td and creating array of results

In [222]:
links = My_table.findAll('td')

extracting text from the tag "td"

In [223]:
Table_content = []
for link in links:
    Table_content.append(link.text)
    


In [224]:
Table_content = [w.replace('\n', '') for w in Table_content]

##  Usig the numpy array of the table so extracted, we create the relevant dataframe required

In [225]:
import numpy as np

Table_content = np.asarray(Table_content)

In [226]:
import pandas as pd
df = pd.DataFrame()

df['PostalCode'] = Table_content[::3]
df['Borough'] = Table_content[1::3]
df['Neighborhood'] = Table_content[2::3]

In [227]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping un-assigned "Borough"

In [228]:
df = df.drop(df[df['Borough'] == 'Not assigned'].index)

In [229]:
df = df.reset_index(drop=True)

Grouping based on "PostalCode" and rest columns are appended using "," as seperator

In [230]:
df1 = df.astype(str).groupby('PostalCode').agg(','.join).reset_index()

## Handling the duplicate substring in "Borough"

In [231]:
import re

In [232]:
bor_array = df1['Borough'].values

In [233]:
result = []

In [234]:
for bor in bor_array:
    result.append(','.join(set(bor.split(','))))


In [235]:
df1['Borough'] = result

##  Handling "Neighborhood" who are not assigned

In [236]:
df1.loc[df1['Neighborhood'] == 'Not assigned', ['Neighborhood']] = df1['Borough']

In [237]:
df1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Printing the shape of the final processed dataframe

In [238]:
df1.shape

(103, 3)